# モデル評価（k-fold法）（練習用）

In [1]:
# Breast cancer wisconsin dataset (classification).
import pandas as pd
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

In [2]:
print(X.shape)
print(y.shape)

(569, 30)
(569,)


holdoutのためのデータ分割はtrain_test_split関数が、kfoldの実行はcross_val_score関数が担います。<br>まずは比較のためholdoutによる評価を実行。

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

# 比較のためholdoutの準備
X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                               test_size=0.2,
                                               random_state=1)
# パイプラインのセット
pipe_logistic = Pipeline([('scl',StandardScaler()),
                          ('est',LogisticRegression(random_state=1))])
# 学習&評価
pipe_logistic.fit(X_train,y_train)       ## trainデータで学習
print('CV_Test:%.6f'%accuracy_score(y_test,pipe_logistic.predict(X_test)))      ## testデータで評価

CV_Test:0.982456


In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(455, 30)
(455,)
(114, 30)
(114,)


In [6]:
## 結果”CV_Test:0.982456”は、testデータに対して

次に、k=10のk-fold法によって異なる10個のCV検証スコアを取得します。実装は非常にシンプルで以下1行でできます。データ分割の処理を明示的に書く必要がない点に留意ください。渡しているデータはXとyです。

In [4]:
from sklearn.model_selection import cross_val_score

cv_results = cross_val_score(pipe_logistic,                  ## モデルインスタンス
                             X,                              ## k-fold法を適用する全データ
                             y.as_matrix().ravel(),          ## k-fold法を適用する全データ
                             cv=10,                          ## 分割数（作成されるモデル数）
                             scoring='accuracy')             ## 出力する評価指標
print(cv_results)

[0.98275862 0.98275862 0.98245614 0.98245614 0.98245614 0.98245614
 0.94736842 1.         1.         0.98214286]


C:\Users\yuki.shimada\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


sklearn.model_selection.cross_val_score
- http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
- 注）k-fold法によってのスコアのみを確認できる関数

In [5]:
## verboseによって途中経過を出力してみる
cross_val_score(pipe_logistic, X, y.as_matrix().ravel(), cv=10, scoring='accuracy', verbose=3)

[CV]  ................................................................
[CV] ....................... , score=0.9827586206896551, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.9827586206896551, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.9824561403508771, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.9824561403508771, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.9824561403508771, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.9824561403508771, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.9473684210526315, total=   0.0s
[CV]  

C:\Users\yuki.shimada\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


array([0.98275862, 0.98275862, 0.98245614, 0.98245614, 0.98245614,
       0.98245614, 0.94736842, 1.        , 1.        , 0.98214286])

CV検証スコアの平均値は、例えば、以下のように取得できます。

In [6]:
print(cv_results.mean(),'+-', cv_results.std())

0.9824853080978307 +- 0.013590418300527833


In [7]:
## 要約統計量を出してみる
pd.DataFrame(cv_results).describe()

,0
count,10.000000
mean,0.982485
std,0.014326
min,0.947368
25%,0.982456
50%,0.982456
75%,0.982759
max,1.000000


以上から、CV検証スコアの平均は0.98程度、標準偏差は0.014と小さいことがわかりました。実務では、CV検証スコアの平均値を評価すれば十分なケースが多いですが、保守的に評価する必要がある場合、例えば、平均値ー標準偏差の序列でベストモデルを評価するなどしましょう。

In [27]:
## 複数のモデルや、そのハイパーパラメータの設定をk-fold法で試し、ベストなスコアのモデルとハイパーパラメータを採用する